<a href="https://colab.research.google.com/github/Nawrin2k16/Dungeon_Quest/blob/main/Dungeon_Quest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keyboard numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.1 MB/s eta 0:00:00


In [2]:
from types import EllipsisType
import numpy as np
import random
import time
import heapq

# Define movement directions
directions = {
    'u': (-1, 0),  # up
    'd': (1, 0),   # down
    'l': (0, -1),  # left
    'r': (0, 1)    # right
}

def reinforcement_learning(grid, start, goal, episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
    # Q-table initialization
    q_table = {}
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            q_table[(i, j)] = {direction: 0 for direction in directions}

    # Action selection policy
    def choose_action(state):
        if random.uniform(0, 1) < epsilon:
            # Explore: random action
            return random.choice(list(directions.keys()))
        # Exploit: best action from Q-table
        #print ("Choose Action: ", max(q_table[state], key=q_table[state].get))
        return max(q_table[state], key=q_table[state].get)

    # Training episodes
    for episode in range(episodes):
        state = start
        while state != goal:
            action = choose_action(state)
            dx, dy = directions[action]
            next_state = (state[0] + dx, state[1] + dy)

            # Check for valid move within grid bounds and not into obstacles
            if (0 <= next_state[0] < grid.shape[0] and 0 <= next_state[1] < grid.shape[1] and grid[next_state] != 1):
                reward = 10 if next_state == goal else -1  # Reward for reaching goal, penalty otherwise
                # Update Q-value using the Bellman equation
                max_future_q = max(q_table[next_state].values())
                q_table[state][action] += alpha * (reward + gamma * max_future_q - q_table[state][action])
                state = next_state
            else:
                # Apply a small penalty for invalid moves
                q_table[state][action] -= 1
        #print("Q table: ", q_table)

    # Function to determine the next position for the monster
    def get_next_position(monster_pos):
        # Choose the best action based on learned Q-table
        best_action = max(q_table[monster_pos], key=q_table[monster_pos].get)
        dx, dy = directions[best_action]
        next_pos = (monster_pos[0] + dx, monster_pos[1] + dy)

        # Ensure the move is valid and within bounds
        if 0 <= next_pos[0] < grid.shape[0] and 0 <= next_pos[1] < grid.shape[1] and grid[next_pos] != 1:
            #print("Next position in function get_next_position", next_pos)
            return next_pos
        print("No valid move")
        return monster_pos  # If no valid move, stay in current position

    #print("get_next_position(start): ", get_next_position(start))
    #print("q_table: ", q_table)

    # Return a function to use the trained Q-table for deciding monster's move
    return get_next_position(start)

def heuristic(a, b):
    """Calculate the Manhattan distance heuristic for A*."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_pathfinding(grid, start, goal):
    """A* pathfinding algorithm to find the shortest path from start to goal."""
    open_set = []
    heapq.heappush(open_set, (0, start))  # (cost, position)
    came_from = {}

    g_score = {start: 0}  # Cost from start to the node
    f_score = {start: heuristic(start, goal)}  # Estimated cost from start to goal

    while open_set:
        current = heapq.heappop(open_set)[1]  # Get the node with the lowest f_score

        # Check if we've reached the goal
        if current == goal:
            return reconstruct_path(came_from, current)

        for direction in directions.values():
            neighbor = (current[0] + direction[0], current[1] + direction[1])
            # Check if the neighbor is within bounds and not a wall
            if (0 <= neighbor[0] < grid.shape[0] and
                0 <= neighbor[1] < grid.shape[1] and
                grid[neighbor] != 1):

                tentative_g_score = g_score[current] + 1  # Cost is 1 for each step
                if tentative_g_score < g_score.get(neighbor, float('inf')):
                    # This path to neighbor is better than any previous one
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)

                    if neighbor not in [i[1] for i in open_set]:
                        heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return []  # Return empty path if there is no path to goal

def reconstruct_path(came_from, current):
    """Reconstruct the path from start to goal."""
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path.append(current)
    total_path.reverse()
    return total_path

def bfs_pathfinding(grid, start, target):
    """Perform BFS to find a path from start to target on the grid."""
    queue = [start]
    visited = {start}  # Keep track of visited cells
    came_from = {}  # Track the path for reconstruction

    while queue:
        current = queue.pop(0)
        if current == target:
            return reconstruct_path(came_from, target)

        x, y = current
        for dx, dy in directions.values():
            nx, ny = x + dx, y + dy
            if 0 <= nx < grid.shape[0] and 0 <= ny < grid.shape[1] and (nx, ny) not in visited and grid[nx, ny] != 1:
                visited.add((nx, ny))
                came_from[(nx, ny)] = current
                queue.append((nx, ny))

    return []  # Return empty if no path found

def unoptimized_bfs_pathfinding(grid, start, target):
    queue = [[start]]  # Queue of paths, each path is a list of positions
    visited = {start}  # Keep track of visited cells

    while queue:
        path = queue.pop(0)  # Get the first path in the queue
        current = path[-1]   # Current node is the last in the path

        # Check if we've reached the target
        if current == target:
            return path  # Return the first path found to the target

        # Explore neighbors in random order (could lead to non-optimal paths)
        x, y = current
        neighbors = list(directions.values())
        random.shuffle(neighbors)  # Randomize order to simulate unoptimized behavior
        for dx, dy in neighbors:
            neighbor = (x + dx, y + dy)

            # Check if the neighbor is within bounds and not a wall or visited
            if (0 <= neighbor[0] < grid.shape[0] and
                0 <= neighbor[1] < grid.shape[1] and
                neighbor not in visited and grid[neighbor] != 1):

                visited.add(neighbor)
                queue.append(path + [neighbor])  # Append new path with this neighbor

    return []  # Return empty if no path found

def add_random_obstacles(grid, path):
    """Add random obstacles to the grid, avoiding the cells in the specified path."""
    num_walls = int(grid.size * 0.2)  # 20% of the grid cells are walls
    for _ in range(num_walls):
        x, y = random.randint(0, grid.shape[0] - 1), random.randint(0, grid.shape[1] - 1)
        if grid[x, y] == 0 and (x, y) not in path:  # Place wall if cell is empty and not on the path
            grid[x, y] = 1  # 1 represents a wall


def print_grid(grid):
    symbols = {0: "__", 1: "##", 2: "P ", 3: "T ", 4: "M "}

    # Top border
    print(" " + "__" * grid.shape[1])

    for row in grid:
        # Side border and row content
        print("|" + "".join(f"|{symbols[cell]}" for cell in row) + "|")

    # Bottom border
    print(" " + "__" * grid.shape[1])

def move_player(grid, player_pos):
    last_valid_position = player_pos  # Store the last valid position
    while True:
        direction = input("Enter direction (u=up, d=down, l=left, r=right): ").strip().lower()
        if direction not in directions:
            print("Invalid input! Please use 'u', 'd', 'l', or 'r'.")
            continue

        # Calculate new position based on direction
        new_pos = (player_pos[0] + directions[direction][0], player_pos[1] + directions[direction][1])

        # Check if the new position is valid (inside grid and not a blockage)
        if (0 <= new_pos[0] < grid.shape[0] and 0 <= new_pos[1] < grid.shape[1]):
            if (grid[new_pos] == 0 or grid[new_pos] == 3):
                grid[player_pos] = 0   # Mark old position as empty
                grid[new_pos] = 2      # Move player to new position
                last_valid_position = new_pos  # Update last valid position
                return new_pos

            elif (grid[new_pos] == 4):
                print("Player jumped on the monster and attempted suicude!")
                return -1
        else:
            print("Invalid Move! Please try again.")
            print(f"Current Position: {last_valid_position}")  # Show last valid position

def move_monster(grid, monster_pos, player_pos, treasure_pos, level = 1):
    """Move the monster towards the nearest target (player or treasure)."""
    if (level == 1):
        path_to_player = unoptimized_bfs_pathfinding(grid, monster_pos, player_pos)
        if treasure_pos:
            path_to_treasure = unoptimized_bfs_pathfinding(grid, monster_pos, treasure_pos)
        print("APPLYING BFS")
    elif (level == 2):
        path_to_player = a_star_pathfinding(grid, monster_pos, player_pos)
        if treasure_pos:
            path_to_treasure = a_star_pathfinding(grid, monster_pos, treasure_pos)
        print("APPLYING A*")
    else:
        path_to_player = reinforcement_learning(grid, monster_pos, player_pos)
        if treasure_pos:
            path_to_treasure = reinforcement_learning(grid, monster_pos, treasure_pos)

    # Select the shortest valid path (player or treasure)
    if treasure_pos and len(path_to_player) > len(path_to_treasure):
        path = path_to_treasure
    elif path_to_player:
        path = path_to_player
    else:
        # If there is no valid path to player or treasure, move randomly
        empty_neighbors = []
        for dx, dy in directions.values():
            nx, ny = monster_pos[0] + dx, monster_pos[1] + dy
            if 0 <= nx < grid.shape[0] and 0 <= ny < grid.shape[1] and grid[nx, ny] == 0:
                empty_neighbors.append((nx, ny))

        if empty_neighbors:
            new_pos = random.choice(empty_neighbors)  # Move to any random valid neighbor
            grid[monster_pos] = 0
            grid[new_pos] = 4
            return new_pos, False  # Monster moved randomly

        # If no valid neighbors, keep the monster in place
        return monster_pos, False

    # If path exists and has more than one step, move monster
    if path and len(path) > 1:
        if level < 3:
            new_pos = path[1]  # Move one step along the path
        else:
            new_pos = path  # If level is 1, move directly to the next step in path

        print("Position in monster move: ", new_pos)

        # Ensure monster only moves to empty spaces or player/treasure positions
        if grid[new_pos] == 0 or grid[new_pos] == 2 or grid[new_pos] == 3:
            grid[monster_pos] = 0
            grid[new_pos] = 4
            return new_pos

        print("Position in monster move: ", new_pos)

    return monster_pos  # If no valid move, return current position


def place_treasure_in_empty_cell(grid):
    empty_cells = [(i, j) for i in range(grid.shape[0]) for j in range(grid.shape[1]) if grid[i, j] == 0]
    if empty_cells:
        return random.choice(empty_cells)
    return None

In [3]:
def check_game_status(player_pos, monsters_pos, treasure_pos, moves_counter, total_moves):
    """Check if the game has reached a win or game-over state."""
    print(f"Moves counter: {moves_counter} and Total Moves: {total_moves}")

    # Check if player ran out of moves
    if moves_counter >= total_moves:
        print("Player ran out of moves! Game over!")
        return -1  # Game over with player running out of moves

    # Check if player reached the treasure
    elif moves_counter < total_moves and player_pos == treasure_pos:
        return 1  # Game over with player win

    # Check if any monster caught the player
    for monster_pos in monsters_pos:
        if monster_pos == player_pos:
            print("Monster caught the player! Game over!")
            return -1  # Game over with monster catching player

    # Check if any monster reached the treasure before the player
    for monster_pos in monsters_pos:
        if monster_pos == treasure_pos:
            print("Monster reached the treasure! Game over!")
            return -1  # Game over with monster reaching treasure

    return False  # Game continues

def create_dungeon_grid(size, level):
    """Create a dungeon grid with a guaranteed path between the player and treasure."""
    grid = np.zeros((size, size), dtype=int)

    # Positions for player and treasure
    player_pos = (0, 0)
    treasure_pos = (size - 1, size - 1)
    grid[player_pos] = 2  # Mark player position
    grid[treasure_pos] = 3  # Mark treasure position

    # Ensure a path from player to treasure using BFS
    path1 = unoptimized_bfs_pathfinding(grid, player_pos, treasure_pos)

    # Mark path on grid (for testing, could be removed in final version)
    for cell in path1:
        if grid[cell] == 0:  # Leave player and treasure positions as-is
            grid[cell] = 0  # Keep path cells empty

    # Place monsters based on level (level defines how many monsters)
    monsters_pos = []
    for _ in range(level):  # Add one monster for each level
        empty_positions = [(i, j) for i in range(size) for j in range(size) if grid[i, j] == 0]
        monster_pos = random.choice(empty_positions)
        grid[monster_pos] = 4  # Mark monster position
        monsters_pos.append(monster_pos)

    # Ensure valid paths for all monsters
    for monster_pos in monsters_pos:
        path2 = unoptimized_bfs_pathfinding(grid, monster_pos, treasure_pos)
        for cell in path2:
            if grid[cell] == 0:  # Leave monster and treasure positions as-is
                grid[cell] = 0  # Keep path cells empty

    # Now add random obstacles without blocking the path
    add_random_obstacles(grid, set(path1 + path2))

    return grid, player_pos, treasure_pos, monsters_pos

def shift_obstacles(grid, player_pos, treasure_pos, monsters_pos, size):
    original_grid = grid.copy()
    # Create a new grid and re-add obstacles while ensuring valid paths for player and monsters
    add_random_obstacles(grid, [player_pos, treasure_pos] + monsters_pos)  # Retain valid positions
    # Ensure paths are still valid for player and all monsters
    if not (unoptimized_bfs_pathfinding(grid, player_pos, treasure_pos) and
            all(unoptimized_bfs_pathfinding(grid, monster_pos, player_pos) for monster_pos in monsters_pos)):
        print("No valid path found after shifting obstacles.")
        grid = original_grid
    return grid

def move_all_monsters(grid, monsters_pos, player_pos, treasure_pos, level):
    """Move all monsters towards the nearest target (player or treasure)."""
    new_monster_positions = []
    for monster_pos in monsters_pos:
        new_pos = move_monster(grid, monster_pos, player_pos, treasure_pos, level)
        new_monster_positions.append(new_pos)
    return new_monster_positions

def main():
    size = 0
    print("Welcome to the dungeon game! Use 'u' for up, 'd' for down, 'l' for left, 'r' for right.")
    while True:
        size = input("Enter the size of dungeon between 3 and 100: ")
        if size.isdigit():  # Check if input is a positive number
            size = int(size)
            if 3 <= size <= 100:
                break  # Exit the loop if size is valid
            else:
                print("Invalid input! Please enter a number between 3 and 100.")
        else:
            print("Invalid input! Please enter a numeric value.")

    level = 1
    while level >= 1:
        flag = True
        print(f"Starting Level {level}!")
        grid, player_pos, treasure_pos, monsters_pos = create_dungeon_grid(size, level)
        treasure_disappear_counter = random.randint(1, size // 2)
        treasure_reappear_counter = random.randint(1, size // 3)


        print_grid(grid)
        moves_counter = 0
        while True:
            player_pos = move_player(grid, player_pos)
            if player_pos == -1:
                print("Game over!")
                flag = False
                break
            moves_counter += 1
            total_moves = size * size // level

            if (moves_counter % size == 0):  # After every 'size' number of player moves
                print("Shifting obstacles...")
                grid = shift_obstacles(grid, player_pos, treasure_pos, monsters_pos, size)
            print("Player moved:")
            print_grid(grid)

            if check_game_status(player_pos, monsters_pos, treasure_pos, moves_counter, total_moves):
                print(f"Level {level} completed! Moving to Level {level+1}")
                level += 1  # Increase level
                size *= 2  # Double the grid size for the next level
                break

            # Treasure disappear and reappear logic
            if treasure_pos:
                treasure_disappear_counter -= 1
                if treasure_disappear_counter <= 0:
                    grid[treasure_pos] = 0  # Remove treasure from grid
                    treasure_pos = None  # Treasure is now gone
                    treasure_reappear_counter = random.randint(1, size // 3)  # Reset reappear counter
                    print(f"Treasure Disappearance Counter has reached 0.\nThe treasure has disappeared! \nPlayer, continue to save yourself from the monster!\nUntil the treasure reappears, Don't die! \nTreasure Reappearance Counter: {treasure_reappear_counter}")
                else:
                    print(f"Treasure Disappearance Counter: {treasure_disappear_counter}")
            else:
                treasure_reappear_counter -= 1
                if treasure_reappear_counter <= 0:
                    treasure_pos = place_treasure_in_empty_cell(grid)
                    if treasure_pos:
                        grid[treasure_pos] = 3  # Place treasure in new empty cell
                    treasure_disappear_counter = random.randint(1, size // 2)  # Reset disappear counter
                    print(f"Treasure Reappearance Counter has reached 0. \nPlayer, Congratulations on surviving with no hope! \nThe treasure has reappeared! \nTreasure Disappearance Counter: {treasure_disappear_counter}")
                else:
                    print(f"Treasure Reappearance Counter: {treasure_reappear_counter}")


            monsters_pos = move_all_monsters(grid, monsters_pos, player_pos, treasure_pos, level)
            print("Monsters moved:")
            print_grid(grid)

            if check_game_status(player_pos, monsters_pos, treasure_pos, moves_counter, total_moves) == -1:
                flag = False
                break

            print(f"Remaining Moves: {total_moves - moves_counter}")
            print(f"Dungeon Shift counter: {size - moves_counter % size}")

            time.sleep(0.5)  # Short delay for readability

        if not flag:
            print("Press any key to continue or press 'n' to exit")
            choice = input().lower()
            if choice == 'n':
                print("Thanks for playing! Goodbye!")
                break
            else:
                level = 1
                while True:
                    size = input("Enter the size of dungeon between 3 and 100: ")
                    if size.isdigit():  # Check if input is a positive number
                        size = int(size)
                        if 3 <= size <= 100:
                            break  # Exit the loop if size is valid
                        else:
                            print("Invalid input! Please enter a number between 3 and 100.")
                    else:
                        print("Invalid input! Please enter a numeric value.")


In [5]:

main()

Welcome to the dungeon game! Use 'u' for up, 'd' for down, 'l' for left, 'r' for right.
Enter the size of dungeon between 3 and 100: 5
Starting Level 1!
 __________
||P |__|__|__|__|
||__|__|__|##|__|
||__|M |__|__|__|
||__|__|__|__|##|
||##|__|__|__|T |
 __________
Enter direction (u=up, d=down, l=left, r=right): r
Player moved:
 __________
||__|P |__|__|__|
||__|__|__|##|__|
||__|M |__|__|__|
||__|__|__|__|##|
||##|__|__|__|T |
 __________
Moves counter: 1 and Total Moves: 25
Treasure Disappearance Counter has reached 0.
The treasure has disappeared! 
Player, continue to save yourself from the monster!
Until the treasure reappears, Don't die! 
Treasure Reappearance Counter: 1
APPLYING BFS
Position in monster move:  (1, 1)
Monsters moved:
 __________
||__|P |__|__|__|
||__|M |__|##|__|
||__|__|__|__|__|
||__|__|__|__|##|
||##|__|__|__|__|
 __________
Moves counter: 1 and Total Moves: 25
Remaining Moves: 24
Dungeon Shift counter: 4
Enter direction (u=up, d=down, l=left, r=right): r
Pla